## Technical interview
By Joost van der Linden, February 11th, 2019.

### Part 1: Data preprocessing
Tasks covered in this notebook: data transformation.

In [1]:
import pandas as pd

data_dir = '../data'

In [2]:
# To auto-reload scripts from /src:
%load_ext autoreload
%autoreload 1

import os
import sys
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

% aimport my_utils
import my_utils

### 1.1 Data ingestion

In [1]:
def read_adult_data(file, skiprows = []):
    '''
    Reads in the Adult Data Set with hard-coded column names. 
    Skipping initial rows if needed, and stripping all strings.
    '''

    # Hard-coded column names, in order of appearance
    adult_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 
                     'marital-status', 'occupation', 'relationship', 'race', 'sex', 
                     'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 
                     'income']

    return (pd.read_csv(file, index_col = False, header = None, 
                        names = adult_columns, skiprows = skiprows, na_values = ' ?')
              .applymap(lambda s: s.strip() if type(s) is str else s))

In [4]:
df_adult_train = read_adult_data(data_dir + '/input/adult-train.csv')
df_adult_test  = read_adult_data(data_dir + '/input/adult-test.csv', skiprows = [0])

**Sanity checks**

In [5]:
df_adult_train.tail(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [6]:
df_adult_test.tail(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.
16280,35,Self-emp-inc,182148,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,60,United-States,>50K.


- The format of `income` is different

In [7]:
len(df_adult_train)

32561

In [8]:
# Extended .describe()
my_utils.sanity_check(df_adult_train)

,dtypes,nan-count,zero-count,count,mean,std,min,25%,50%,75%,max
age,int64,0,0,32561,38.5816,13.6404,17,28,37,48,90
workclass,object,1836,0,-,-,-,-,-,-,-,-
fnlwgt,int64,0,0,32561,189778,105550,12285,117827,178356,237051,1.48470e+06
education,object,0,0,-,-,-,-,-,-,-,-
education-num,int64,0,0,32561,10.0807,2.57272,1,9,10,12,16
marital-status,object,0,0,-,-,-,-,-,-,-,-
occupation,object,1843,0,-,-,-,-,-,-,-,-
relationship,object,0,0,-,-,-,-,-,-,-,-
race,object,0,0,-,-,-,-,-,-,-,-
sex,object,0,0,-,-,-,-,-,-,-,-


- `age` values are sensible
- most `capital-gain` and `capital-loss` values are zero
- `capital-gain seems` to be capped at 100k 
- `hours-per-week` seems to be capped at 100 hours

In [9]:
# Same checks for df_adult_test 
len(df_adult_test)

16281

In [10]:
my_utils.sanity_check(df_adult_test)

,dtypes,nan-count,zero-count,count,mean,std,min,25%,50%,75%,max
age,int64,0,0,16281,38.7675,13.8492,17,28,37,48,90
workclass,object,963,0,-,-,-,-,-,-,-,-
fnlwgt,int64,0,0,16281,189436,105715,13492,116736,177831,238384,1.4904e+06
education,object,0,0,-,-,-,-,-,-,-,-
education-num,int64,0,0,16281,10.0729,2.56755,1,9,10,12,16
marital-status,object,0,0,-,-,-,-,-,-,-,-
occupation,object,966,0,-,-,-,-,-,-,-,-
relationship,object,0,0,-,-,-,-,-,-,-,-
race,object,0,0,-,-,-,-,-,-,-,-
sex,object,0,0,-,-,-,-,-,-,-,-


**`income` fix**

In [11]:
df_adult_test['income'] = df_adult_test['income'].str.replace('.', '')

In [12]:
df_adult_test['income'].tail(3)

16278    <=50K
16279    <=50K
16280     >50K
Name: income, dtype: object

**Graduation year**

In [13]:
df_adult_train['estimated-graduation-year'] = my_utils.add_graduation_year(df_adult_train)
df_adult_test['estimated-graduation-year']  = my_utils.add_graduation_year(df_adult_test)

In [14]:
df_adult_train[['age', 'education', 'estimated-graduation-year']].sample(3)

,age,education,estimated-graduation-year
29069,58,Some-college,1957
19484,47,HS-grad,1965
1821,31,7th-8th,1977


**Net capital gain/loss**

In [15]:
df_adult_train['net-capital'] = df_adult_train['capital-gain'] - df_adult_train['capital-loss']
df_adult_test['net-capital']  = df_adult_test['capital-gain'] - df_adult_test['capital-loss']

In [16]:
df_adult_train['net-capital'].describe()

count    32561.000000
mean       990.345014
std       7408.986951
min      -4356.000000
25%          0.000000
50%          0.000000
75%          0.000000
max      99999.000000
Name: net-capital, dtype: float64

#### College educated
`education-num` is ordered from 1 (preschool) to 10 (some college) to 16 (Doctorate)

In [17]:
df_adult_train['college-educated'] = (df_adult_train['education-num'] >= 10)
df_adult_test['college-educated']  = (df_adult_test['education-num'] >= 10)

In [18]:
df_adult_train['college-educated'].value_counts()

True     17807
False    14754
Name: college-educated, dtype: int64

### 1.2 Serialize dataframes
For convenient loading in subsequent notebooks.

In [19]:
df_adult_train.to_pickle(data_dir + '/processed/adult-train-preprocessed.pickle')
df_adult_test.to_pickle(data_dir + '/processed/adult-test-preprocessed.pickle')